In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
import datetime


In [ ]:
df = pd.read_csv("cleaned_data.csv")
df_copy = df.copy()
def data_preprocessing(data, current_year = 2025):
    """
    data: pd.DataFrame
    
    Output
    processed_data:pd.DataFrame
    """
    #Handle negative values in "no_of_employees"
    data["no_of_employees"] = np.abs(data["no_of_employees"])
    data = data.drop("case_id", axis = 1)

    #Feature engineering
    def prev_wage_norm(data):
        if data["unit_of_wage"] == "Year":
            return data["prevailing_wage"]
        elif data["unit_of_wage"] == "Month":
            return data["prevailing_wage"] * 12
        elif data["unit_of_wage"] == "Week":
            return data["prevailing_wage"] * 52
        else:
            return data["prevailing_wage"] * 40 * 52
    
    data["Prev_wage_normal"] = data.apply(prev_wage_norm, axis = 1)
    data["age_of_estab"] = current_year - data["yrs_of_estab"]

    def quality_assessment(data):
        if data["has_job_experience"] == "Y" and data["requires_job_training"] == "Y":
            return 4
        elif data["has_job_experience"] == "Y" and data["requires_job_training"] == "N":
            return 3
        elif data["has_job_experience"] == "N" and data["requires_job_training"] == "Y":
            return 2
        else:
            return 1
    data["Quality_Rating"]  = data.apply(quality_assessment, axis = 1)
    data["education_of_employees"] = data["education_of_employees"].map({"High School": 0, "Bachelor's": 1, "Master's": 2, "Doctorate": 3})
    data["continent"] = data["continent"].map({"South America": 0, "North America": 1, "Oceania": 2, "Asia": 3, "Africa": 4, "Europe": 5})

    return data



    

In [20]:
df[df["no_of_employees"] <= 0]

,case_id,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status
245,EZYV246,Europe,Master's,N,N,-25,1980,Northeast,39452.9900,Year,Y,Certified
378,EZYV379,Asia,Bachelor's,N,Y,-11,2011,Northeast,32506.1400,Year,Y,Denied
832,EZYV833,South America,Master's,Y,N,-17,2002,South,129701.9400,Year,Y,Certified
2918,EZYV2919,Asia,Master's,Y,N,-26,2005,Midwest,112799.4600,Year,Y,Certified
6439,EZYV6440,Asia,Bachelor's,N,N,-14,2013,South,103.9700,Hour,Y,Denied
6634,EZYV6635,Asia,Bachelor's,Y,N,-26,1923,West,5247.3200,Year,Y,Denied
7224,EZYV7225,Europe,Doctorate,N,N,-25,1998,Midwest,141435.9500,Year,Y,Certified
7281,EZYV7282,Asia,High School,N,N,-14,2000,Midwest,58488.5000,Year,Y,Denied
7318,EZYV7319,Asia,Bachelor's,Y,Y,-26,2006,South,115005.6100,Year,Y,Certified
7761,EZYV7762,Asia,Master's,N,N,-11,2009,Midwest,38457.5100,Year,Y,Certified


In [ ]:
df[""]